Fit of mock data using a two flavor neutrino oscillation framework

Similar to https://arxiv.org/pdf/1707.07081.pdf


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize

I use the functions that were provided in the github file

In [2]:
def atmo_event_rate(energy, cos_theta, sin2theta=0.9, dm2=2.4e-3):
    flux = 1e5*energy**-1.7 # the true neutrino flux is steeper, this takes into account 
                            # neutrino cross-section, which is proportional to E
                            # as well as that we are working with logarithmic energy bins    
    l = 12e3 * cos_theta # base line - max corresponds to diameter of the Earth 12e3 km

    # two flavor oscillation propability for muon disappearance - see wikipedia 
    osc_prop = 1-sin2theta * np.sin(1.27*dm2*l/energy) 

    return flux*osc_prop

def run_experiment(energy, cosdec, sin2theta=0.9, dm2=2.4e-3):
    x = np.array(energy.size)
    events=np.zeros(energy.size*cosdec.size)

    i = 0
    for e in energy:
        for c in cosdec:
            mu = atmo_event_rate(e,c,sin2theta,dm2)
            sig = np.sqrt(mu)
            event_random = np.random.normal(mu,sig)
            events[i] = int(event_random) 
            
            i=i+1
                   
    return events

In [3]:
# lets generate one experiment, with 10 energy bins and 10 cos(declination) bins

cosdec =  np.linspace(0, 1, 10) # 10 steps in cos(declination) from 0 to 1 
log10_energy = np.linspace(1, 2, 10) # 10 steps in log10(energy) from 10 to 100 GeV
energy=10**log10_energy

# run the experiment 
events=run_experiment(energy,cosdec,0.9,2.4e-3)

# lets print the events per bin

i = 0
for e in energy:
    for c in cosdec:
        print("energy, cosdec, events:",e,c, events[i])
        i=i+1

energy, cosdec, events: 10.0 0.0 1960.0
energy, cosdec, events: 10.0 0.1111111111111111 1287.0
energy, cosdec, events: 10.0 0.2222222222222222 700.0
energy, cosdec, events: 10.0 0.3333333333333333 282.0
energy, cosdec, events: 10.0 0.4444444444444444 206.0
energy, cosdec, events: 10.0 0.5555555555555556 410.0
energy, cosdec, events: 10.0 0.6666666666666666 791.0
energy, cosdec, events: 10.0 0.7777777777777777 1519.0
energy, cosdec, events: 10.0 0.8888888888888888 2168.0
energy, cosdec, events: 10.0 1.0 2958.0
energy, cosdec, events: 12.91549665014884 0.0 1350.0
energy, cosdec, events: 12.91549665014884 0.1111111111111111 916.0
energy, cosdec, events: 12.91549665014884 0.2222222222222222 637.0
energy, cosdec, events: 12.91549665014884 0.3333333333333333 346.0
energy, cosdec, events: 12.91549665014884 0.4444444444444444 206.0
energy, cosdec, events: 12.91549665014884 0.5555555555555556 118.0
energy, cosdec, events: 12.91549665014884 0.6666666666666666 186.0
energy, cosdec, events: 12.915

Now I define a function to get the above values in an array

In [4]:
def fit_arrays():
    
    

# lets generate one experiment, with 10 energy bins and 10 cos(declination) bins

    cosdec =  np.linspace(0, 1, 10) # 10 steps in cos(declination) from 0 to 1 
    log10_energy = np.linspace(1, 2, 10) # 10 steps in log10(energy) from 10 to 100 GeV
    energy=10**log10_energy

# run the experiment 
    events=run_experiment(energy,cosdec,0.9,2.4e-3)

# lets print the events per bin

    i = 0
    energy_array=np.zeros(0)
    cosdec_array=np.zeros(0)
    events_array=np.zeros(0)

    for e in energy:
        for c in cosdec:
            energy_array=np.append([energy_array],[e])
            cosdec_array=np.append([cosdec_array],[c])
            events_array=np.append([events_array],[events[i]])
        
        #print("energy, cosdec, events:",e,c, events[i])
            i=i+1
        
    return energy_array, cosdec_array, events_array

In [5]:
fit_daten=fit_arrays()
fit_daten

(array([ 10.        ,  10.        ,  10.        ,  10.        ,
         10.        ,  10.        ,  10.        ,  10.        ,
         10.        ,  10.        ,  12.91549665,  12.91549665,
         12.91549665,  12.91549665,  12.91549665,  12.91549665,
         12.91549665,  12.91549665,  12.91549665,  12.91549665,
         16.68100537,  16.68100537,  16.68100537,  16.68100537,
         16.68100537,  16.68100537,  16.68100537,  16.68100537,
         16.68100537,  16.68100537,  21.5443469 ,  21.5443469 ,
         21.5443469 ,  21.5443469 ,  21.5443469 ,  21.5443469 ,
         21.5443469 ,  21.5443469 ,  21.5443469 ,  21.5443469 ,
         27.82559402,  27.82559402,  27.82559402,  27.82559402,
         27.82559402,  27.82559402,  27.82559402,  27.82559402,
         27.82559402,  27.82559402,  35.93813664,  35.93813664,
         35.93813664,  35.93813664,  35.93813664,  35.93813664,
         35.93813664,  35.93813664,  35.93813664,  35.93813664,
         46.41588834,  46.41588834,  46.

In [7]:
energy=fit_daten[0]
cos_theta=fit_daten[1]
y=fit_daten[2]  # y is the monte carlo data of flux*probability

Now I make a fit

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


import scipy 
import scipy.optimize as so


from scipy import odr
from uncertainties import unumpy, ufloat

In [10]:
def func(p, X):
    sin2theta,dm2  = p
    energy, cos_theta= X
    return atmo_event_rate(energy, cos_theta, sin2theta, dm2)


quad_model = scipy.odr.Model(func)
data = scipy.odr.RealData((energy,cos_theta),y)
odr = scipy.odr.ODR(data, quad_model, beta0=[0.9, 2.4e-3])
odr.set_job(fit_type=2)
out = odr.run()
popt = out.beta
perr = out.sd_beta
print("fit parameter 1-sigma error")
print("———————————–")
for i in range(len(popt)):
    print(str(popt[i])+" +- "+str(perr[i]*1/np.sqrt(out.res_var)))

fit parameter 1-sigma error
———————————–
0.8929769178773765 +- 0.00017957797308037262
0.0023848740304321316 +- 2.1435662636348374e-07


The first value is sin2theta. The second value is dm2

Note that the uncertainty is the uncertainty from the fit. Now I want to calculate the uncertainty of the parameters through the Cramer-Rao bound

The Cramer Rao bound is defined by $$V\geq\frac{1}{<\frac{\partial log(L)}{\partial a}>}$$

With $$L=\prod_{i=1}^{n}P_i$$

and $$P=1-sin^2(2\theta)\cdot sin^2\Big(1.27\cdot \frac{(\Delta\cdot M)^2 L}{E}\Big)$$

Consequently we have $$log(L)=\sum_{i=1}^{n}log(P_i)$$

and $$\frac{\partial log(L)}{\partial a}=\sum_{i=1}^{n}\frac{\partial log(P_i)}{\partial a}$$

In our case a will be sin2theta and dm2

Now I write a function to calculate the derivative numerically.

In [15]:
from scipy.misc import derivative

In [20]:
def probability_sin2theta(sin2theta,energy, cos_theta, dm2):
    l = 12e3 * cos_theta
    return np.log(1-sin2theta * np.sin(1.27*dm2*l/energy))    #calculate log(P)

def probability_dm2(dm2,energy, cos_theta, sin2theta):
    l = 12e3 * cos_theta
    return np.log(1-sin2theta * np.sin(1.27*dm2*l/energy))  #calculate log(P)

def Cramer_Rao(energy_array,costheta_array,sin2theta=0.9,dm2=2.4e-3):
    
    sin2theta_array=np.zeros(0)
    dm2_array=np.zeros(0)
    
    for i in range(len(energy_array)):  #calculate P_i and the derivative for each energy and costheta
        sin2theta_Cramer_Rao=derivative(probability_sin2theta, sin2theta, dx=1e-6,args=((energy_array[i],costheta_array[i],dm2)))
        
        dm2_Cramer_Rao=derivative(probability_dm2, dm2, dx=1e-6,args=((energy_array[i],costheta_array[i],sin2theta)))
        
        sin2theta_array=np.append([sin2theta_array],[sin2theta_Cramer_Rao])
        dm2_array=np.append([dm2_array],[dm2_Cramer_Rao])
        
    return sin2theta_array, dm2_array   #calculate the derivatives for sin2theta and dm2 




In [21]:
Cramer_Rao(energy,cos_theta,sin2theta=0.9,dm2=2.4e-3)

(array([ 0.        , -0.61361306, -2.09642369, -6.05465359, -9.85186704,
        -4.61525425, -1.54698997, -0.39693027,  0.09958404,  0.34168097,
         0.        , -0.42898545, -1.25163171, -2.98757405, -6.63220314,
        -9.99968564, -6.55754968, -2.94779789, -1.23340732, -0.41987043,
         0.        , -0.30812215, -0.80918826, -1.67209476, -3.24089662,
        -6.03965185, -9.43649192, -9.16340665, -5.67977994, -3.02722305,
         0.        , -0.22558919, -0.55109435, -1.03621273, -1.78584797,
        -2.98110411, -4.88396639, -7.56548594, -9.81278283, -9.25004102,
         0.        , -0.16746907, -0.38871216, -0.68635184, -1.09516758,
        -1.6691619 , -2.49110001, -3.67837806, -5.35334558, -7.47525244,
         0.        , -0.12559783, -0.28094281, -0.4750612 , -0.7204666 ,
        -1.03468001, -1.44236229, -1.97805109, -2.68881644, -3.63388025,
         0.        , -0.09491414, -0.20661557, -0.33884401, -0.4963949 ,
        -0.68545966, -0.91407806, -1.19272735, -1.5

And then we can calculate the bound. For sin2theta I get

In [22]:
1/(np.sum(Cramer_Rao(energy,cos_theta,sin2theta=0.9,dm2=2.4e-3)[0]))**2

2.8295547306305676e-05

For dm2 I get

In [23]:
1/(np.sum(Cramer_Rao(energy,cos_theta,sin2theta=0.9,dm2=2.4e-3)[1]))**2

1.648651120363176e-08

Now I want to see how often the fit value will be in the 1 sigma region.

In [24]:
sigma_sin2theta=np.sqrt(2.8295547306305676e-05)
sigma_sin2theta

0.005319355910851019

In [25]:
sigma_dm2=np.sqrt(1.648651120363176e-08)
sigma_dm2

0.00012839980998284912

In [27]:
def func(p, X):
    sin2theta,dm2  = p
    energy, cos_theta= X
    return atmo_event_rate(energy, cos_theta, sin2theta, dm2)

In [39]:
def repeat_experiment(N):
    
    N_R=0  
    
    for i in range(N):
        
    
        fit_daten=fit_arrays()
        fit_daten
    
    
        energy=fit_daten[0]
        cos_theta=fit_daten[1]
        y=fit_daten[2]  # y is the monte carlo data of flux*probability
    
    
    
        #def func(p, X):
        #    sin2theta,dm2  = p
        #    energy, cos_theta= X
        #return atmo_event_rate(energy, cos_theta, sin2theta, dm2)


        quad_model = scipy.odr.Model(func)
        data = scipy.odr.RealData((energy,cos_theta),y)
        odr = scipy.odr.ODR(data, quad_model, beta0=[0.9, 2.4e-3])
        odr.set_job(fit_type=2)
        out = odr.run()
        popt = out.beta
        perr = out.sd_beta
        #print("fit parameter 1-sigma error")
        #print("———————————–")
        #for i in range(len(popt)):
         #   print(str(popt[i])+" +- "+str(perr[i]*1/np.sqrt(out.res_var)))
        
        if popt[0]>=0.9-sigma_sin2theta and popt[0]<=0.9+sigma_sin2theta and popt[1]>=2.4e-3-sigma_dm2 and popt[1]<=2.4e-3+sigma_dm2:
            N_R=N_R+1
    
    return N_R
        
    
    
    
    

In [40]:
repeat_experiment(100)

81

The fit values are inside the errors 81 times